In [34]:
import pymongo
import pandas as pd
from pandas import Series,DataFrame
import matplotlib as plt
import matplotlib.dates as mdate
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#二行即可搞定画图中文乱码
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
pd.set_option('display.max_columns', None)  # 设置显示最大行

#连接数据库
client = pymongo.MongoClient('localhost',27017)
futures = client.futures
marketdata = futures.marketdata
unit = futures.unit
position = futures.position

date='20180101'
#加载数据
marketdata = DataFrame(list(marketdata.find({'date': {'$gte':date}})))
unit = DataFrame(list(unit.find()))
position = DataFrame(list(position.find({'date': {'$gte': date}})))

# #列名重名
# position= position.rename(columns={'var':'variety'})

#删除/选取某行含有特殊数值的列

position=position.dropna()

# ##选择需要显示的字段
print(marketdata)
position=position[['date','variety','symbol','long_openIntr','short_openIntr']]

#修改日期格式，非常方便
# marketdata['date'] = pd.to_datetime(marketdata['date'], format='%Y/%m/%d')
# position['date'] = pd.to_datetime(position['date'], format='%Y/%m/%d')
# 值替换replace()
# position=position.replace(['PTA'],'TA')
#汇总合约
netPosition=position.groupby(['date','variety'])[['long_openIntr','short_openIntr']].sum()

netPosition['净持仓']=netPosition.apply(lambda x:x['long_openIntr']-x['short_openIntr'],axis=1)
netPosition['上一日净持仓']=netPosition.groupby('variety')['净持仓'].shift(1)
netPosition['净持仓变化量']=netPosition.apply(lambda x: x['净持仓']-x['上一日净持仓'],axis=1)
netPosition=netPosition.dropna()

# marketdata.set_index(['date','variety'])
#计算价格变化量


print(marketdata.head())
print(netPosition.head())
# 并集
# trade=pd.merge(netPosition,marketdata, on=['date','variety'],how='outer')
# trade = trade.dropna(axis=0, how='any')
# trade['交易信号'] = trade.apply(lambda x: 0 if x['净持仓变化量']*x['change']>=0  else 1 if x['净持仓变化量']>0 else -1,axis=1)
# trade=trade.set_index('date')
# trade = trade[trade['交易信号'] != 0]
# del trade['上一日净持仓']
# print(trade.head())


# trade.to_csv(r"c:\futures1.csv",mode='a',encoding='ANSI',header=True)

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
                  long_openIntr  short_openIntr      净持仓   上一日净持仓  净持仓变化量
date     variety                                                         
20180410 A              87606.0        103538.0 -15932.0 -21032.0  5100.0
         AG            249916.0        254001.0  -4085.0  -7516.0  3431.0
         AL            257039.0        296998.0 -39959.0 -41884.0  1925.0
         AP             36056.0         32587.0   3469.0   1248.0  2221.0
         AU             94792.0         30458.0  64334.0  65115.0  -781.0
